# Part 1: Large-Scale Batch Analysis (IMDb)

## Objective
The first phase of this project focuses on Batch Processing. We aim to ingest, clean, and analyze a large relational dataset—the IMDb Non-Commercial Dataset—to extract meaningful insights about movies, actors, and production trends over the last century.

## Technical Approach: Why Polars?
Handling multiple large TSV files (some exceeding several gigabytes) requires an efficient computing engine. Instead of using Pandas (which loads everything into RAM eagerly) or PySpark (which requires a complex Java/Hadoop setup), we utilize Polars.

**Key advantages for this project:**
1.  **Lazy Evaluation (scan_csv):** Polars builds a query execution plan and only processes data when a result is explicitly requested (.collect()). This allows us to handle datasets larger than the available RAM.
2.  **Performance:** Written in Rust, Polars is multi-threaded and significantly faster than traditional tools for filtering and joining operations.

## Workflow
1.  **Ingestion:** Defining a schema-on-read strategy to handle raw TSV files and null values (\N).
2.  **Cleaning:** Casting data types (Integers, Dates) and handling missing values.
3.  **Analysis:** Answering specific business questions through complex joins across 5 different tables (basics, ratings, names, crew, akas).

In [3]:
%pip install polars seaborn matplotlib

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 0.0/802.4 kB ? eta -:--:--
   ---------------------------------------- 802.4/802.4 kB 11.5 MB/s  0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   -- ------------------------------------- 2.4/44.5 MB 12.3 MB/s eta 0:00:04
   ---- ----------------------------------- 5.0/44.5 MB 12.1 MB/s eta 0:00:04
   ------ --------------------------------- 7.3/44.5 MB 11.6 MB/s eta 0:00:04
   -------- ------------------------------- 9.7/44.5 MB 11.6 MB/s eta 0:00:03
   ----------- ---------------------------- 12.3/44.5 MB 11.7 MB/s eta 0:00:03
   ------------- -------------------------- 14.9/44.5 MB 11.7 MB/s eta 0:00:03
   --------------- ------------------------ 17.6/44

In [3]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

pl.Config.set_tbl_rows(10)


def load_imdb_lazy(filename):
    return pl.scan_csv(
        filename,
        separator='\t',
        null_values=['\\N'],
        quote_char=None,
        ignore_errors=True
    )


q_basics  = load_imdb_lazy('data/title.basics.tsv')
q_ratings = load_imdb_lazy('data/title.ratings.tsv')
q_names   = load_imdb_lazy('data/name.basics.tsv')
q_crew    = load_imdb_lazy('data/title.crew.tsv')
q_akas    = load_imdb_lazy('data/title.akas.tsv')
q_names = q_names.with_columns(pl.col("birthYear").cast(pl.Int32, strict=False))


q_basics = q_basics.with_columns([
    pl.col("startYear").cast(pl.Int32, strict=False),
    pl.col("runtimeMinutes").cast(pl.Int32, strict=False)
])

q_ratings = q_ratings.with_columns([
    pl.col("averageRating").cast(pl.Float64, strict=False),
    pl.col("numVotes").cast(pl.Int32, strict=False)
])


1. How many total people in data set?

In [4]:
total_people = q_names.select(pl.len()).collect().item()
print(f"Total number of people in dataset: {total_people}")

Total number of people in dataset: 14942131


2. What is the earliest year of birth?

In [ ]:
earliest_year = q_names.select(pl.col("birthYear").min()).collect().item()
print(f"Earliest birth year listed: {earliest_year}")

Earliest birth year listed: 4


3. How many years ago was this person born?

In [9]:
from datetime import datetime

current_year = datetime.now().year
years_ago = current_year - earliest_year
print(f"This person was born {years_ago} years ago.")

This person was born 2021 years ago.


4. Determine if this date of birth is correct (Code & Reasoning)

In [ ]:
suspicious_person = (
    q_names
    .filter(pl.col("birthYear") == earliest_year)
    .select(["nconst", "primaryName", "birthYear", "deathYear", "primaryProfession", "knownForTitles"])
    .collect()
)
print(suspicious_person)

shape: (1, 6)
┌───────────┬────────────────────┬───────────┬───────────┬───────────────────┬─────────────────────┐
│ nconst    ┆ primaryName        ┆ birthYear ┆ deathYear ┆ primaryProfession ┆ knownForTitles      │
│ ---       ┆ ---                ┆ ---       ┆ ---       ┆ ---               ┆ ---                 │
│ str       ┆ str                ┆ i32       ┆ i64       ┆ str               ┆ str                 │
╞═══════════╪════════════════════╪═══════════╪═══════════╪═══════════════════╪═════════════════════╡
│ nm0784172 ┆ Lucio Anneo Seneca ┆ 4         ┆ 65        ┆ writer            ┆ tt0043802,tt0218822 │
│           ┆                    ┆           ┆           ┆                   ┆ ,tt0049203,…        │
└───────────┴────────────────────┴───────────┴───────────┴───────────────────┴─────────────────────┘


Validity Analysis: The result indicates a birth year of 4 (AD), which would make this person approximately 2021 years old.

Is this date correct? Strictly speaking, in the context of a living film professional, this date is incorrect. However, within the context of the IMDb database structure, it is likely a valid entry for a historical figure, but misleading regarding active participation in film production.

5. What is the most recent date of birth?

In [ ]:
most_recent_year = q_names.select(pl.col("birthYear").max()).collect().item()
print(f"Most recent birth year: {most_recent_year}")

Most recent birth year: 2025


6. What percentage of the people do not have a listed date of birth?

In [ ]:
percentage_missing = (
    q_names.select(
        (pl.col("birthYear").null_count() / pl.len()) * 100
    )
    .collect()
    .item()
)
print(f"Percentage of people with no birth year: {percentage_missing:.2f}%")

Percentage of people with no birth year: 95.58%


7. What is the length of the longest "short" after 1900?

In [13]:
longest_short = (
    q_basics
    .filter(
        (pl.col("titleType") == "short") & 
        (pl.col("startYear") > 1900)
    )
    .select(pl.col("runtimeMinutes").max())
    .collect()
    .item()
)
print(f"Length of the longest short film after 1900: {longest_short} minutes")

Length of the longest short film after 1900: 1311 minutes


8. What is the length of the shortest "movie" after 1900?

In [14]:
shortest_movie = (
    q_basics
    .filter(
        (pl.col("titleType") == "movie") & 
        (pl.col("startYear") > 1900)
    )
    .select(pl.col("runtimeMinutes").min())
    .collect()
    .item()
)
print(f"Length of the shortest movie after 1900: {shortest_movie} minutes")

Length of the shortest movie after 1900: 1 minutes


9. List of all of the genres represented

In [ ]:
all_genres = (
    q_basics
    .select(pl.col("genres").str.split(","))
    .explode("genres")
    .filter(pl.col("genres").is_not_null())
    .select(pl.col("genres").unique())
    .sort("genres")
    .collect()
)

print("List of all represented genres:")
print(all_genres["genres"].to_list())

List of all represented genres:
['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']


10. What is the highest rated comedy "movie"? (Tie broken by votes)

In [ ]:
best_comedy = (
    q_basics.join(q_ratings, on="tconst")
    .filter(
        (pl.col("titleType") == "movie") &
        (pl.col("genres").str.contains("Comedy")) 
    )
    .sort(["averageRating", "numVotes"], descending=[True, True])
    .head(1)
    .collect()
)

print("Highest rated comedy movie:")
print(best_comedy.select(["primaryTitle", "startYear", "averageRating", "numVotes"]))

best_comedy_id = best_comedy[0, "tconst"]

Highest rated comedy movie:
shape: (1, 4)
┌──────────────┬───────────┬───────────────┬──────────┐
│ primaryTitle ┆ startYear ┆ averageRating ┆ numVotes │
│ ---          ┆ ---       ┆ ---           ┆ ---      │
│ str          ┆ i32       ┆ f64           ┆ i32      │
╞══════════════╪═══════════╪═══════════════╪══════════╡
│ O La La      ┆ 2018      ┆ 10.0          ┆ 6        │
└──────────────┴───────────┴───────────────┴──────────┘


11. Who was the director of the movie?

In [ ]:
directors_names = (
    q_crew
    .filter(pl.col("tconst") == best_comedy_id)
    .select(pl.col("directors").str.split(","))
    .explode("directors")
    .join(q_names, left_on="directors", right_on="nconst")
    .select("primaryName")
    .collect()
)

print(f"Director(s) of the movie: {directors_names['primaryName'].to_list()}")

Director(s) of the movie: ['Sripad Pai']


12. List, if any, the alternate titles for the movie

In [ ]:
alternate_titles = (
    q_akas
    .filter(pl.col("titleId") == best_comedy_id)
    .select(["title", "region", "language"])
    .collect()
)

print("Alternate titles:")
print(alternate_titles)

Alternate titles:
shape: (2, 3)
┌─────────┬────────┬──────────┐
│ title   ┆ region ┆ language │
│ ---     ┆ ---    ┆ ---      │
│ str     ┆ str    ┆ str      │
╞═════════╪════════╪══════════╡
│ O La La ┆ null   ┆ null     │
│ O La La ┆ IN     ┆ en       │
└─────────┴────────┴──────────┘


# Part 2: Real-Time Streaming Architecture

## Objective and Context
After analyzing historical "cold" data (Batch Processing) in the first part using the IMDb dataset, we are now moving to the analysis of "hot" data in real-time (Stream Processing).

The goal is to simulate a data monitoring pipeline capable of reacting instantly to external events.

## Technical Architecture
To achieve this, we connect to the Wikimedia EventStreams API, which broadcasts all Wikipedia edits in real-time via the SSE (Server-Sent Events) protocol.

Our pipeline follows this logic:
1.  **Ingestion (Source):** A persistent connection to the Wikimedia recentchange stream.
2.  **Filter:** Isolation of events concerning our entities of interest (e.g., *Leonardo DiCaprio, Inception*) and high-traffic streams (*Sandbox*) for testing purposes.
3.  **Processing:**
    - **Standard Metrics:** Every relevant edit is cleaned and stored in a structured file (CSV) for future statistics.
    - **Alert System:** Conditional logic detects if the author of the edit is a Bot. If so, the event is flagged as an "Alert" and routed to a specific storage file (JSON) for auditing.

This simplified architecture allows us to combine analytical storage and event-driven reaction.

In [ ]:
%pip install sseclient-py requests

In [ ]:
import json
import csv
import os
import time
import requests
from datetime import datetime
from sseclient import SSEClient

TRACKED_KEYWORDS = [
    "DiCaprio",
    "Nolan",
    "Inception",
    "Action",
    "IMDb",
    "Sandbox",   # Will match any sandbox page
    "Wikipédia"  # Will match French Wikipedia edits
]

METRICS_FILE = "wiki_metrics.csv"
ALERTS_FILE = "wiki_alerts.json"
WIKI_STREAM_URL = "https://stream.wikimedia.org/v2/stream/recentchange"

def init_storage():
    with open(METRICS_FILE, mode='w', newline='', encoding='utf-8') as f:
        csv.writer(f).writerow(["timestamp", "entity", "user", "is_bot", "length_diff", "comment"])
    
    with open(ALERTS_FILE, mode='w', encoding='utf-8') as f:
        f.write("[]")

def save_metric(data):
    try:
        with open(METRICS_FILE, mode='a', newline='', encoding='utf-8') as f:
            csv.writer(f).writerow([
                datetime.fromtimestamp(data['timestamp']),
                data['title'],
                data['user'],
                data['bot'],
                data['length']['new'] - data['length']['old'],
                data['comment'][:50]
            ])
        print(f"\033[92m[SAVED] {data['title']}\033[0m")
    except Exception as e:
        print(f"Error saving: {e}")

def trigger_alert(data):
    try:
        with open(ALERTS_FILE, 'r', encoding='utf-8') as f:
            content = f.read()
            alerts = json.loads(content) if content else []
    except:
        alerts = []

    alerts.append({
        "alert_type": "BOT_ACTION",
        "alert_timestamp": str(datetime.now()),
        "event_data": data
    })

    with open(ALERTS_FILE, 'w', encoding='utf-8') as f:
        json.dump(alerts, f, indent=4)
    print(f"\033[91m[ALERT] Bot Action Logged!\033[0m")

def run_stream_job(duration_seconds=60):
    print(f"🚀 Starting Stream... Waiting for events...")
    init_storage()
    start_time = time.time()
    
    headers = {'User-Agent': 'IMDb_Project_Bot/1.0 (contact@example.com)'}
    
    try:
        response = requests.get(WIKI_STREAM_URL, stream=True, headers=headers)
        client = SSEClient(response)
        
        event_count = 0
        
        for event in client.events():
            if time.time() - start_time > duration_seconds:
                print("\n⏱️ Finished.")
                break
            
            if event.event == 'message':
                try:
                    change = json.loads(event.data)
                except ValueError:
                    continue
                
                title = change.get('title', '')
                is_bot = change.get('bot', False)
                event_count += 1
                
                match = False
                for keyword in TRACKED_KEYWORDS:
                    if keyword.lower() in title.lower():
                        match = True
                        break
                
                if match:
                    save_metric(change)
                    if is_bot:
                        trigger_alert(change)
                else:
                    if event_count % 5 == 0:
                        print(f"   (Ignored: {title})")
                        
    except KeyboardInterrupt:
        print("\n🛑 Stopped.")
    except Exception as e:
        print(f"\n❌ Error: {e}")

run_stream_job(60)

🚀 Starting Stream... Waiting for events...
   (Ignored: Category:PD NASA)
   (Ignored: Category:Files from NASA HQ Flickr stream)
   (Ignored: File:Tombe Ramsès V Ramsès VI Vallée Rois - Louxor (EG) - 2025-12-10 - 125.jpg)
   (Ignored: Category:Creative Commons Attribution-Share Alike 4.0 missing SDC copyright license)
   (Ignored: File:ST-22 (31814310837).jpg)
   (Ignored: File:HeliRussia 2010 (303-13).jpg)
   (Ignored: Category:Noindexed pages)
   (Ignored: File:HeliRussia 2011 (365-20).jpg)
   (Ignored: Castration)
   (Ignored: User talk:Hemant500us)
   (Ignored: Thảo luận Thành viên:203.205.0.84)
   (Ignored: Q4246156)
   (Ignored: File:HeliRussia 2010 (303-14).jpg)
   (Ignored: Category:Digital Public Library of America files missing required SDC statements)
   (Ignored: Category:Artworks without Wikidata item)
   (Ignored: Q4674340)
   (Ignored: File:Strasbourg-Statue de Jean Hultz (5).jpg)
   (Ignored: Q83242410)
   (Ignored: Category:MM7322 (aicraft))
   (Ignored: Төркем:Геогра

In [1]:
import polars as pl
import json
import os

print("1. METRICS (wiki_metrics.csv)")
if os.path.exists("wiki_metrics.csv"):
    df_metrics = pl.read_csv("wiki_metrics.csv")
    print(df_metrics.head(5))
    print(f"\nTotal events captured: {len(df_metrics)}")
else:
    print("File not found.")

print("\n" + "="*50 + "\n")

print("2. ALERTS (wiki_alerts.json)")
if os.path.exists("wiki_alerts.json"):
    with open("wiki_alerts.json", "r", encoding="utf-8") as f:
        alerts = json.load(f)

    print(f"Total Alerts (Bot Actions): {len(alerts)}")

    if len(alerts) > 0:
        print("\nExample of the last alert triggered:")
        print(json.dumps(alerts[-1], indent=2))
else:
    print("File not found.")

1. METRICS (wiki_metrics.csv)
shape: (5, 6)
┌────────────┬───────────────────────┬──────────────┬────────┬─────────────┬───────────────────────┐
│ timestamp  ┆ entity                ┆ user         ┆ is_bot ┆ length_diff ┆ comment               │
│ ---        ┆ ---                   ┆ ---          ┆ ---    ┆ ---         ┆ ---                   │
│ str        ┆ str                   ┆ str          ┆ bool   ┆ i64         ┆ str                   │
╞════════════╪═══════════════════════╪══════════════╪════════╪═════════════╪═══════════════════════╡
│ 2025-12-19 ┆ User:SLMn475/sandbox  ┆ SLMn475      ┆ false  ┆ 0           ┆ null                  │
│ 18:15:57   ┆                       ┆              ┆        ┆             ┆                       │
│ 2025-12-19 ┆ File:Field Guns In    ┆ فيصل         ┆ false  ┆ 190         ┆ /*                    │
│ 18:16:05   ┆ Action at S…          ┆              ┆        ┆             ┆ wbsetlabel-add:1|ar   │
│            ┆                       ┆         